## Multi Agent-Systems


 A team is a group of agents that work together to achieve a common goal

In [ ]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)
# model_client_2 = 

# Single Agent Approach

A single agent is going to create a short story for us

In [2]:
from autogen_agentchat.agents import (AssistantAgent)

story_agent = AssistantAgent(
    name = 'story_agent',
    model_client=model_client,
    system_message='You are a creating writer. Generate a short story about a brave knight and a dragon.'
)

In [4]:
from autogen_agentchat.messages import TextMessage

async def test_simple_agent():
    task = TextMessage(
        content='Write a short story about a brave knight and a dragon. Keep it up to 50 words',
        source='user'
    )
    result = await story_agent.run(task=task)
    print(result.messages[-1].content)

await test_simple_agent()

In a land of whispers, Sir Elric faced the feared dragon Zephyr atop Mistpeak. Rather than battle, they spoke, discovering shared loneliness. Zephyr guarded the kingdom thereafter, tales of violence dissolved into ones of friendship, where fire and steel forged a bond stronger than any blade.


In a land of whispers, Sir Elric faced the feared dragon Zephyr atop Mistpeak. Rather than battle, they spoke, discovering shared loneliness. Zephyr guarded the kingdom thereafter, tales of violence dissolved into ones of friendship, where fire and steel forged a bond stronger than any blade.

# Multi Agent Team Approach

We will be using 3 agents to now write the story, each with its own work and mastery

In [5]:
plot_agent = AssistantAgent(
    name = 'plot_writer',
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on the Pokemon's journey"
)

character_agent = AssistantAgent(
    name = 'character_writer',
    model_client=model_client,
    system_message="You develop characters. Describe the pokemon and the villian in detail, including their motivations and backgrounds."
)

ending_agent = AssistantAgent(
    name = 'ending_writer',
    model_client=model_client,
    system_message="You wrute engaging endings. conclude the story with a twist."
)


RoundRobinGroupChat is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

In [9]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants= [plot_agent, character_agent, ending_agent],
    max_turns=6
)
team.component_type

'team'

In [10]:

async def test_team():
    task = TextMessage(
        content='Write a short story a brave boy and his Pokemon. Keep it up to 50 words',
        source='user'
    )

    result = await team.run(task=task)
    for each_agent_message in result.messages:
        print(f'{each_agent_message.source} : {each_agent_message.content}')
        

await test_team()

user : Write a short story a brave boy and his Pokemon. Keep it up to 50 words
plot_writer : In the dense Misty Forest, brave Leo and his Pikachu, Sparky, sought the mystical Thunder Stone. Navigating eerie paths, they encountered a fierce Beedrill swarm. Sparky's Thunderbolt cleared a path, revealing the gleaming stone. Victorious and bonded, the duo returned as heroes, ready for future quests.
character_writer : In the heart of Misty Forest, courageous Leo and his Pikachu, Sparky, ventured forth. Entangled in vines, a Beedrill threatened. Sparky's electric charge flashed brilliantly, freeing them. Victorious, they uncovered the mystical Thunder Stone. United, the pair’s bond strengthened, eager for adventures to come.
ending_writer : Just as they grasped the Thunder Stone, Leo felt a peculiar jolt. In a startling twist, Sparky didn't evolve; instead, Leo felt an electrifying surge of power. Now imbued with Pikachu-like speed and senses, Leo realized their relationship wasn't just tra

user : Write a short story a brave boy and his Pokemon. Keep it up to 50 words



1. plot_writer : In Lumina Town, young Kai and his fiery Charmander ventured into the Whispering Woods. The pair faced the ominous Shadow Grove, where flickering shadows danced. Together, they unleashed a dazzling Flamethrower, dispelling darkness. United by courage and friendship, they emerged victorious, ready for endless adventures ahead.



2. character_writer : In the vibrant world of Lumina Town, Kai's adventurous spirit was matched only by his fiery companion, Charmander. 

**Charmander**: A tiny lizard Pokémon with a flame at the tip of its tail, Charmander is spirited and fiercely loyal. This Charmander bears a unique streak of blue in its flame, symbolizing its unusual potential and heritage. Having grown up in the bustling markets of Lumina Town, it's accustomed to cheer and challenges alike. Charming but determined, Charmander’s motivation is driven by a thirst for growth. It yearns to evolve, not just physically but in strength and wisdom, to protect those it cares about.

Navigating through the lively world, their latest trial lay in the daunting Whispering Woods. At the heart of the woods lurked the villainous Master Umbra.

**Master Umbra**: A shadowy figure cloaked in constant twilight, Umbra was once known as Ardor, a revered Pokémon researcher committed to studying light’s impact on Pokémon evolution. Betrayed and ridiculed by peers for his radical theories, he turned to darkness, embracing the shadows. Master Umbra commands a legion of shadow Pokémon, thriving in dimness he manipulates. His ultimate goal: harnessing darkness to eclipse every region, showing the world power lies not in light, but in the void it resists. Umbra's cunning is overshadowed only by his understanding of light and shadow, but deep down, he desires recognition and validation.

Face-to-face with Master Umbra in the intimidating Shadow Grove, Kai and Charmander find themselves tested. Shadows swirl, threatening to overwhelm, but fueled by bravery and the symbolic blue flame, Charmander releases a scorching Flamethrower. Light pierces darkness, driving back Umbra’s forces.

Unfazed, Umbra vanishes with a vow to reclaim dominion, leaving Kai and Charmander that much closer to their dream: to stand as protectors and champions of balance, ensuring darkness never eclipses the light. 



3. ending_writer : As Kai and Charmander celebrated their victory over Master Umbra, the burnt shadows began to swirl and reform, revealing a hidden message in the grove's ashes. It read: "Look deeper, young one." Intrigued, Kai followed the cryptic hint, discovering an ancient stone tablet with the truth that Umbra, once known as Ardor, was his estranged grandfather. The twist of fate was a reminder that sometimes, the shadows we face are closer than they appear, casting new light on his path ahead.

# Comparision Table

## Single-Agent vs. Multi-Agent Comparison

| Aspect             | Single Agent                | Multi-Agent Team              |
|--------------------|-----------------------------|-------------------------------|
| **Creativity**     | Limited to one viewpoint    | Diverse ideas from each agent |
| **Depth**          | Basic story elements        | Detailed plot, characters, ending |
| **Flexibility**    | Stuck to one style          | Adaptable with specialized roles |
| **Setup Effort**   | Simple, one agent           | More agents, team setup       |

In [11]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 19, 18, 59, 15, 46020, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=101) metadata={} created_at=datetime.datetime(2025, 6, 19, 18, 59, 18, 857256, tzinfo=datetime.timezone.utc) content="Leaves of amber softly dance,  \nUnderneath the crisp fall glance.  \nThe whispering wind tells a tale,  \nOf harvest moons and shadowed trails.  \n\nPumpkin scents in the cool, brisk air,  \nSweaters wrapped with loving care.  \nA symphony of rust and gold,  \nAs nature’s canvas does unfold.  \n\nBare trees stand with quiet grace,  \nIn the season's warm embrace.  \nThrough the cycles, echoes call,  \nWelcoming the heart of fall.  \n" type='TextMessage'
source='character_writer' models_usage=RequestUsage(prompt_tokens=149, completion_tokens=502) metadata={} created_at=datetime.datetime

In [12]:
from autogen_agentchat.teams import RoundRobinGroupChat

team2 = RoundRobinGroupChat(
    participants= [plot_agent],
    max_turns=2
)

In [13]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team2.reset()  # Reset the team for a new task.
async for message in team2.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 19, 19, 1, 18, 987796, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=111) metadata={} created_at=datetime.datetime(2025, 6, 19, 19, 1, 22, 958212, tzinfo=datetime.timezone.utc) content="In the season where the leaves set sail,  \nVibrant hues tell a colorful tale.  \nWhispers of wind in a cooling breeze,  \nUndress the branches with gentle ease.    \n\nPumpkins gather in fields of gold,  \nWarmth of cider against the cold.  \nCrisp air dances, rustling trees,  \nAutumn's symphony, nature’s tease.  \n\nA tapestry of orange and brown,  \nNature dons its grandest gown.  \nIn the fall's embrace we find,  \nA fleeting beauty, intertwined.  " type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=149, completion_tokens=647) metadata={} created_at=date

In [14]:
message

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 19, 19, 1, 18, 987796, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='plot_writer', models_usage=RequestUsage(prompt_tokens=34, completion_tokens=111), metadata={}, created_at=datetime.datetime(2025, 6, 19, 19, 1, 22, 958212, tzinfo=datetime.timezone.utc), content="In the season where the leaves set sail,  \nVibrant hues tell a colorful tale.  \nWhispers of wind in a cooling breeze,  \nUndress the branches with gentle ease.    \n\nPumpkins gather in fields of gold,  \nWarmth of cider against the cold.  \nCrisp air dances, rustling trees,  \nAutumn's symphony, nature’s tease.  \n\nA tapestry of orange and brown,  \nNature dons its grandest gown.  \nIn the fall's embrace we find,  \nA fleeting beauty, intertwined.  ", type='TextMessage'), TextMessage(source='plot_writer', models_usage=RequestUsa

In [15]:
from autogen_agentchat.teams import RoundRobinGroupChat

team3 = RoundRobinGroupChat(
    participants= [plot_agent],
    # max_turns=2
)

In [ ]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team3.reset()  # Reset the team for a new task.
async for message in team3.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 19, 19, 9, 18, 365921, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=103) metadata={} created_at=datetime.datetime(2025, 6, 19, 19, 9, 21, 780708, tzinfo=datetime.timezone.utc) content="Leaves of amber gently fall,  \nDancing to the autumn's call.  \nWhispers of the cooling breeze,  \nWeaving through the golden trees.\n\nPumpkins gleam in sunset hues,  \nBeneath a sky of deeper blues.  \nSweaters warm against the chill,  \nAs twilight wraps the quiet hill.\n\nCrisp and sweet, the air does beam,  \nPainting worlds that softly dream.  \nIn fall's embrace, nature's art,  \nA season's song within the heart.  " type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=141, completion_tokens=1) metadata={} created_at=datetime.datetime(2025, 6, 19, 19, 9,